In [ ]:

from pydantic import BaseModel
from typing import Any, Optional
from unstructured.partition.pdf import partition_pdf
 
# Load 
path = "reference/finance/"
file = "351.pdf"
# Get elements
raw_pdf_elements = partition_pdf(
    filename=path+file,
    extract_images_in_pdf=True,
    infer_table_structure=True, 
    # Post processing to aggregate text once we have the title 
    chunking_strategy="by_title",
    # Chunking params to aggregate text blocks
    # Require maximum chunk size of 4000 chars
    # Attempt to create a new chunk at 3800 chars
    # Attempt to keep chunks > 2000 chars 
    max_characters=4000, 
    new_after_n_chars=3800, 
    combine_text_under_n_chars=2000,
    image_output_dir_path=path
)

class Element(BaseModel):
    type: str
    text: Any
 
# Categorize by type
categorized_elements = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(Element(type="table", text=str(element)))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        categorized_elements.append(Element(type="text", text=str(element)))
 
# Tables
table_elements = [e for e in categorized_elements if e.type == "table"]
print(table_elements) 
# output: 28 elements in the PDF file
 
# Text
text_elements = [e for e in categorized_elements if e.type == "text"]
print(text_elements) 
# output: 127 elements in the PDF file

In [3]:
import requests

def call_local_llm(url, role, prompt):
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }

    params = {
        "message_self": True  # 設定傳入的訊息參數
    }

    data = [
        {"role": "system", "content": role},
        {"role": "user", "content": prompt},
    ]

    response = requests.post(url, headers=headers, params=params, json=data)
    return response.text

call_local_llm('http://127.0.0.1:8087/chat','你是一個來自台灣的AI助理，你的名字是 TAIDE，樂於以台灣人的立場幫助使用者，會用正體中文回答問題，擅長幫助改寫','你好，我想要查詢這份文件的內容。')

'你好！很高興為你服務。你想要我幫你看哪一份文件呢？請你告訴我，我將很樂意為你解說。'

In [ ]:
def rewrite_content(content_type: str, original_text: str, style: str = "正式") -> str:
    """
    content_type: 要改寫的內容類型 (文件/報告/郵件等)
    original_text: 原始文字
    style: 改寫風格 (正式/口語/簡潔等)
    """
    
    # 建立提示詞模板
    prompt_template = f"""
    請幫我改寫以下{content_type}內容，使用{style}的風格：

    原始內容：
    {original_text}

    要求：
    1. 保持原意
    2. 調整用詞與語氣
    3. 提升表達的清晰度
    """
    
    # 設定系統角色提示詞
    system_prompt = """你是一個專業的文字改寫助理，擅長：
    1. 調整文字表達方式
    2. 優化句子結構
    3. 讓內容更容易理解
    4. 維持原始文意
    請根據使用者的需求提供改寫建議。
    """
    
    # 調用 LLM API
    response = call_local_llm(
        url='http://127.0.0.1:8087/chat',
        role=system_prompt,
        prompt=prompt_template
    )
    
    return response

# 使用範例
document_text = "這份文件描述了我們公司的營運狀況..."
result = rewrite_content(
    content_type="公司報告",
    original_text=document_text,
    style="正式"
)
print(result)

In [6]:
def enhance_question(original_question: str, context: str = "", focus: str = "一般"):
    """
    original_question: 原始問題
    context: 問題背景/上下文 (選填)
    focus: 問題重點方向 (一般/技術/商業等)
    """
    
    # 問題改寫模板
    prompt_template = f"""
    請協助優化以下問題，要求：
    
    原始問題：{original_question}
    
    優化規則：
    1. 輸出不超過 100 字
    2. 使用精準的專業用語
    3. 明確標示關鍵重點
    4. 保持問題簡潔完整
    
    格式要求：
    - 直接輸出優化後的問題
    - 不要加入解釋或說明
    - 使用問號結尾
    - 避免贅字
    """
    
    system_prompt = """你是專業的問題優化助理：
    1. 直接輸出改寫後的問題
    2. 不提供額外解釋
    3. 確保問題明確且具體
    4. 避免過度延伸原始問題範圍
    """

    
    # 調用 LLM API
    response = call_local_llm(
        url='http://127.0.0.1:8087/chat',
        role=system_prompt,
        prompt=prompt_template
    )
    
    return response


question = "匯款銀行及中間行所收取之相關費用由誰負擔?"
enhanced = enhance_question(
    original_question=question,
    context="PDF文件內容分析",
    focus="技術"
)
print(enhanced)

在跨境匯款中，收取費用的責任歸屬為何？關鍵因素包括匯出國銀行、匯入國銀行、貨幣轉換及中間機構。請確認並提出具體銀行名稱及交易細節，以獲得精確的費用資訊。


In [ ]:
import json
import requests
import time

def call_local_llm(url, role, prompt):
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }

    params = {
        "message_self": True  # 設定傳入的訊息參數
    }

    data = [
        {"role": "system", "content": role},
        {"role": "user", "content": prompt},
    ]

    response = requests.post(url, headers=headers, params=params, json=data)
    return response.text
# 讀取 JSON 文件
with open('dataset/preliminary/questions_example.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 定義改寫問題的函數
def enhance_question(original_question: str, context: str = "", focus: str = "一般"):
    prompt_template = f"""
    請協助優化以下問題，要求：
    
    原始問題：{original_question}
    
    優化規則：
    1. 在問題中對時間、主體做大約的描述解釋
    2. 使用精準的專業用語描述
    3. 保持問題簡潔（50字內）
    4. 補充關鍵背景資訊
    5. 避免冗長敘述
    
    格式要求：
    - 直接輸出優化後的問題
    - 使用問號結尾
    - 避免贅字
    """
    
    system_prompt = """你是金融領域的專業問題優化助理：
    1. 直接輸出改寫後的問題
    2. 確保包含完整的時間、主體、事件資訊
    3. 使用準確的金融專業術語
    4. 避免過度延伸原意
    """

    response = call_local_llm(
        url='http://127.0.0.1:8087/chat',
        role=system_prompt,
        prompt=prompt_template
    )
    
    return response

# 改寫所有問題並將改寫後的問題與原來的問題相加


for question in data['questions']:
    if 'query' in question:
        start_time = time.time()
        original_query = question['query']
        enhanced_query = enhance_question(original_query)
        print(f"原始問題：{original_query}")
        print(f"改寫後問題：{enhanced_query}")
        question['query'] = original_query + " " + enhanced_query
        end_time = time.time()
        print(f"改寫問題所需時間: {end_time - start_time} 秒")


# 將改寫後的問題寫入新的 JSON 文件
with open('enhanced_questions.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

print("問題改寫完成並已輸出至 enhanced_questions.json")

原始問題：匯款銀行及中間行所收取之相關費用由誰負擔?
改寫後問題：在國際匯款情境下，涉及委託銀行及清算銀行，相關費用負擔為何？請提供相關背景資訊。 」。


在國際匯款情境下，涉及委託銀行及清算銀行，相關費用負擔為何？請提供相關背景資訊。


在國際匯款過程中，費用負擔通常由委託銀行(客戶銀行)及清算銀行(如SWIFT網路中的參與銀行)共同承擔。這些費用可能包含匯款手續費、外匯轉換費、清算費等，由雙方銀行在交易前協商並由客戶負擔。然而，具體費用結構與金額將取決於銀行間的協議及匯款的具體條件。為提供更精確的資訊，請提供相關背景資訊，如匯款金額、匯款幣別、匯款途徑等。
改寫問題所需時間: 272.455420255661 秒
原始問題：本公司應在效力停止日前多少天以書面通知要保人？
改寫後問題：在不具體提供保險契約及相關細節的情況下，無法精確計算通知受益人截止日之前的天數。然而，根據保險法一般原則，通知應於效力停止日前以書面送達受益人或被保險人。

為了獲得更精確的建議，請提供相關背景資訊，如保險契約類型、效力停止日、受益人姓名及地址。
改寫問題所需時間: 91.52870559692383 秒
原始問題：「完全失能保險金」的受益人是否可以指定或變更?
改寫後問題：在未簽訂專屬契約的情況下，受益人指定或變更是否可行，將取決於保險政策的具體條款及當地法律規定。以美國為例，若政策為定期保險或變額保險，受益人通常可在保單持有人同意下進行變更；但於終身保險中，變更受益人可能較為限制，甚至不可行。

請提供有關該保險政策的類型、簽訂日期、當事人姓名等背景資訊，以釐清受益人指定或變更的可行性。
改寫問題所需時間: 124.67518782615662 秒
原始問題：本契約內容的變更應經由誰同意並批註？
改寫後問題：在關於《2022年一般保險契約》修正的契約變更議題上，請詢問負責監督及批准此類修正的主責委員會名稱及其所屬機構。
改寫問題所需時間: 45.34825420379639 秒
原始問題：「基本保額」是指什麼？
改寫後問題：在不具體提及時間或主體的情況下，基本保額是指保險契約中，為保障被保險人最低限度財務需求，而約定之基本保險金額。這在人壽保險及財產保險中皆常見，是確保基礎保障足以支應基本生活費、債務等基本財務需求的基準。
改寫問題所需時間: 78.3889529705047